In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from keras.models import Sequential

from keras.layers import Dense, Activation, Conv2D, BatchNormalization,MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D,Conv2DTranspose

from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau 

from keras.layers import Input
from keras.layers import concatenate
from keras.models import Model
from keras.layers import Reshape

Using TensorFlow backend.


In [2]:
images = os.listdir("images")

train = [t for t in images if "Train" in t]
test = [t for t in images if "Test" in t]

In [3]:
train_csv = pd.read_csv("train.csv")
test_csv = pd.read_csv("test.csv")


In [4]:
train_images = []

for i in train_csv["image_id"]:
    img = cv2.imread("images/"+i+".jpg")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150,150),interpolation=cv2.INTER_AREA)
    train_images.append(img)

In [5]:
y = train_csv.drop('image_id',axis=1)
y = np.array(y)
y.shape

(1821, 4)

In [6]:
train_images[0].shape

(150, 150, 3)

In [7]:
X = np.array(train_images)

In [8]:
X = X/255.0    
X.shape 

(1821, 150, 150, 3)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train,y_val = train_test_split(X,y,test_size = .20, random_state = 2)

### Apply Smote

In [10]:
from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=42)
# x_smote, y_smote = sm.fit_resample(x_train.reshape(x_train.shape[0], -1), y_train)

In [11]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=42)
x_smote, y_smote = sm.fit_resample(x_train.reshape(x_train.shape[0], -1), y_train)

print('After OverSampling, the shape of train_X: {}'.format(x_smote.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_smote.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_smote==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_smote==0)))

Before OverSampling, counts of label '1': [424  72 490 470]
Before OverSampling, counts of label '0': [1032 1384  966  986] 

After OverSampling, the shape of train_X: (1960, 67500)
After OverSampling, the shape of train_y: (1960, 4) 

After OverSampling, counts of label '1': [490 490 490 490]
After OverSampling, counts of label '0': [1470 1470 1470 1470]


### Model

In [12]:
def construct_model() :
    
    model = Sequential()
    
    #Block 1
    model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu',input_shape=(150,150,3)))
    model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.3))
    
    #Block 2
    
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.3))

    #Block 3
    
    model.add(Conv2D(256,kernel_size=(3,3),padding='valid',activation='relu'))
    model.add(Conv2D(256,kernel_size=(3,3),padding='valid',activation='relu'))
    model.add(Conv2D(256,kernel_size=(3,3),padding='valid',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.2))
    
    #Block 4 
    
    model.add(Conv2D(512,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(512,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(512,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.2))    
    
    
    
    
    
    #Block 5
    
    model.add(Conv2D(1024,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(1024,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.2))

    
    #Final Block
    
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(4,activation='softmax'))
        
    #Compile
    
    model.compile(optimizer='Adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
    
    
    return model  

In [13]:
model = construct_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [14]:
x_smote = x_smote.reshape(x_smote.shape[0], 150, 150, 3)

In [15]:
model.fit(x_smote, y_smote, epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1960/1960 [==============================] - 58s 30ms/step - loss: 5.7368 - acc: 0.2760
Epoch 2/5
1960/1960 [==============================] - 48s 24ms/step - loss: 3.2562 - acc: 0.2827
Epoch 3/5
1960/1960 [==============================] - 48s 25ms/step - loss: 1.5821 - acc: 0.2913
Epoch 4/5
1960/1960 [==============================] - 48s 25ms/step - loss: 1.4274 - acc: 0.3204
Epoch 5/5
1960/1960 [==============================] - 49s 25ms/step - loss: 1.3325 - acc: 0.3862


# Flow from directory

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                  height_shift_range=0.2,
                                  shear_range = 0.2,
                                  horizontal_flip = True)
valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./Data/Train/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=4,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
valid_generator = valid_datagen.flow_from_directory(
    directory=r"./Data/Validation/",
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=4,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
# test_generator = test_datagen.flow_from_directory(
#     directory=r"./Data/Test/",
#     target_size=(150, 150),
#     color_mode="rgb",
#     batch_size=1,
#     class_mode=None,
#     shuffle=False,
#     seed=42
# )

In [ ]:
model = construct_model()
model.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)

history = model.fit_generator(train_generator,
                             steps_per_epoch = STEP_SIZE_TRAIN,
                             epochs = 10,
                             validation_data = valid_generator,
                             validation_steps= STEP_SIZE_VALID,
                            callbacks = [model_checkpoint, early_stopping,reduce_lr])